In [41]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [42]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences

In [43]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [44]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [219]:
target_base_CT_minus(seq)

GAA


['CCTTCCTCAACAGCCTTTTGAA']

In [256]:
def target_base_CT_minus(seq):
    match = re.findall(r'(?=(CC\w{20}))', seq)#ここ不安
    target_list=[]
    for i in range(len(match)):
        target_1 = match[i][19:22]
        target_2 = match[i][18:21]
        target_3 = match[i][17:20]
        if (target_1 =='TCG'or target_1 =='CTG'or target_1 =='TTG')or(target_2 =='TCG'or target_2 =='CTG'or target_2 =='TTG')or(target_3 =='TCG'or target_3 =='CTG'or target_3 =='TTG'):
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [257]:
def target_base_AG_minus(seq):
    match = re.findall('(?=(\w{18,22}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        
        if target =='CCA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [303]:
def CT_target_start_minus(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            seq = match.group()[::-1]#TGGのTの位置を取得したいので逆にしてTの位置を判断している
            add_seq = re.search(('GCT|GTC|GTT'),seq)
            add_num =add_seq.start()
            start_index = match.start()
            result.append(start_index+21-add_num)
    return result

In [306]:
target_gene_start('NM_010294',CT_target_start_minus(target_base_CT_minus(seq),seq))

array([97603043, 97603292, 97603439, 97603451, 97603451, 97603520,
       97603579, 97603828, 97603828, 97603898, 97604011, 97604016,
       97604037, 97604058, 97604081, 97604081, 97604134, 97604343,
       97604389, 97604485, 97604703, 97604820])

In [259]:
transcription('ACACAAGAATACCCAAAAGAAGG')

'CCTTCTTTTGGGTATTCTTGTGT'

In [260]:
seq='CCTTCTTTTGGGTATTCTTGTGT'

In [261]:
target_base_CT_minus(seq)

['CCTTCTTTTGGGTATTCTTGTG']

In [267]:
seq='AAAAACCAGATGAGCCAAGAGTCGATGG'

In [268]:
target_base_AG_minus(seq)

['CCAGATGAGCCAAGAGTCGATGG']

In [271]:
AG_target_start_minus(target_base_AG_minus(seq),seq)

[7]

In [264]:
target_base_CT_minus(base_data['NM_010294::chr5:97602988-97604882'])

['CCTTTATTTTAAGGAGTATTTG',
 'CCATTTTCAGGGTCCTTTGCTG',
 'CCAAACATTTACCCCTTCTGCA',
 'CCCCTTCTGCAGCTCCGGCTGC',
 'CCCTTCTGCAGCTCCGGCTGCC',
 'CCGGAATGTGCAGAATATCTGC',
 'CCATCGACCTGCAAATGACTGA',
 'CCCTGAGGTTGTTATAATTTCG',
 'CCTGAGGTTGTTATAATTTCGA',
 'CCTTCTAAGGCATAAAAAACTG',
 'CCTGTTCCGTATGTGTTTTTGG',
 'CCGTATGTGTTTTTGGCTTGAC',
 'CCTTCATGGAAACACATTTGTC',
 'CCCACTAAAGCAGCAGACTGAT',
 'CCCCCAAACATCCAGATATTGG',
 'CCCCAAACATCCAGATATTGGC',
 'CCATAGATCTCAGAAGAACTGC',
 'CCTTCCTCAACAGCCTTTTGAA',
 'CCACCGCAGCTTCACAGCACTG',
 'CCACAGTAGACTGGGTTCTGAG',
 'CCTTCTTTTGGGTATTCTTGTG',
 'CCAACGGTCCCGCAGAGGTTTG']

In [270]:
def AG_target_start_minus(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            
            result.append(start_index+2)#もしかしたら+3?
    return result

In [272]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [277]:
seq = base_data['NM_010294::chr5:97602988-97604882']

In [278]:
target_gene_start('NM_010294',AG_target_start_minus(target_base_AG_minus(seq),seq))

array([97603613, 97604093, 97604286, 97604496, 97604501, 97604543,
       97604678])

In [10]:
def transcription(seq):
    complementary = {'A':'T','T':'A','G':'C','C':'G'}
    rev_seq = seq[::-1]
    result = []
    for base in rev_seq:
        base =base.upper()
        result.append(complementary[base])
    
    return ''.join(result)

In [11]:
def where_start_codon(name):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsEnd'].to_list()[0])#strand-minusなのでcdsEndが開始コドン
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
            exon_num =s
    return exon_num


In [89]:
def transcription_exon_start(name,sequence_data):#transcription名乗ってるけど翻訳される前のエクソンを出してる。
 
    
    
    data = gene_df[gene_df['name']==name]
    
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)

    result = ''.join(exon_seq_list)
    
    exon_num=where_start_codon(name)
    
    exon_count = data['exonCount']
    exon_start = int(exon_count)-1-exon_num
    
    
    
    if exon_start == 0:
        result = result[:- (int(end_list[len(start_list)-2])-int(data['cdsEnd'].to_list()[0]))]
        

    else:
        exon_start_num =0
        for s in range(exon_start):
            exon_start_num += int(end_list[-2-s])-int(start_list[-2-s])
            
        exon_start_num += int(end_list[-2-exon_start])-int(data['cdsEnd'].to_list()[0])
        result = result[:-(exon_start_num)]
        
    
    return result
    
    

In [90]:
def end_codon(cds_seq):#seqはtranscription
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [96]:
def add_num_list(name):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [130]:
def correct_codon_num(name,end_codon_data,range_list):#txStartからの距離に変換
    
    
    data = gene_df[gene_df['name']==name]
    exonCount = int(data['exonCount'])
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    
    exon_num=where_start_codon(name)
    
    exon_count = data['exonCount']
    exon_start = int(exon_count)-1-exon_num
    

    if exon_start == 0:#確認してない
        add_num =  (int(end_list[len(start_list)-2])-int(data['cdsEnd'].to_list()[0]))
        

    else:#あってそう
        add_num =0
        for s in range(exon_start): 
            add_num += int(end_list[-2-s])-int(start_list[-2-s])  

        
        add_num += int(end_list[-2-exon_start])-int(data['cdsEnd'].to_list()[0])
     
   
    end_codon_data = np.array(end_codon_data)   
    end_codon_data = end_codon_data + add_num
    
    end_codon_data = abs(end_codon_data - range_list[exonCount-1][1]+1)#+1するかは微妙なライン、この後の処理に合わせて変えるべき
    
    result = end_codon_data.tolist()
    
    return result

In [147]:
def codon_in_exon(name,correct_num,range_list):

    data = gene_df[gene_df['name']==name]
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
    
    
    
    indices = []
    for number in correct_num:
        for index, interval in enumerate(range_list):
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
            
    for i in indices:
        if i == 0:
            add_result.append(set_num)
        else:
            add_result.append(int(start[i]))

    return add_result
    

In [141]:
def add_num_correct(name,correct_num, range_list):
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_result=[]
         
     
    indices = []
    for number in correct_num:
        for index, interval in enumerate(range_list):
            if int(interval[0]) <= int(number) < int(interval[1]):
                indices.append(index)
                break
    n =0        
    for i in indices:
        add_result.append(int(correct_num[n])-int(range_list[i][0]))
        n += 1

    return add_result

In [149]:
gene_df[gene_df['geneName']=='Gk2']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
32045,Gk2,NM_010294,chr5,-,97602988,97604882,97603171,97604836,1,"97602988,","97604882,"


In [150]:
'NM_010294::chr5:97602988-97604882'

'NM_010294::chr5:97602988-97604882'

In [151]:
a= end_codon(transcription(transcription_exon_start('NM_010294',base_data['NM_010294::chr5:97602988-97604882'])))

In [152]:
aa= codon_in_exon('NM_010294',correct_codon_num('NM_010294',a,add_num_list('NM_010294')),add_num_list('NM_010294'))

In [153]:
bb= add_num_correct('NM_010294',correct_codon_num('NM_010294',a,add_num_list('NM_010294')),add_num_list('NM_010294'))

In [154]:
np.array(aa)+np.array(bb)

array([97604820, 97604784, 97604718, 97604703, 97604682, 97604673,
       97604559, 97604538, 97604496, 97604478, 97604370, 97604343,
       97604295, 97604286, 97604193, 97604058, 97604037, 97604016,
       97603965, 97603845, 97603722, 97603683, 97603629, 97603623,
       97603569, 97603530, 97603524, 97603422, 97603380, 97603368,
       97603362, 97603329, 97603299])

In [155]:
transcription('ACCAGATGAGCCAAGAGTCGATGG')

'CCATCGACTCTTGGCTCATCTGGT'

In [198]:
#base_data['NM_010294::chr5:97602988-97604882']

In [157]:
transcription(base_data['NM_010294::chr5:97602988-97604882'])

'GTCTCCTAACTGGCATCACCAGGGAAACAGTCCACTGACCAGTTTCATGGCAGCCTCGAAGCAAACCTCTGCGGGACCGTTGGTAGGAGCGGTGGTCCAGGGAACCAACTCCACTCGCTTTCTAGTTTTCAATAGCAAAACAGCAGAACTAGTTTGCTCTCATCAGGTGGAACTGACACAAGAATACCCAAAAGAAGGATGGGTGGAGCAAGACCCCAAGGAAATCCTTAAATCTGTCTATGAGTGCATAGCGAAAGCTTGTGAAAAACTTGCTGAAGTGAACATTGATATTTCCAACATTAAAGCTATCGGCGTGAGTAACCAGCGGGAAACCACTGTGGTCTGGGACAAGTTCACGGGAGACCCTCTGTACAATGCCGTGGTGTGGCTTGATCTCAGAACCCAGTCTACTGTGGAGACTCTCACTAAGAAAATCCCAGGAAATAGCAACTTTGTCAAGTCTAAGACCGGCCTTCCTCTTAGCACTTACTTCAGTGCTGTGAAGCTGCGGTGGATGCTGGACAACCTAAGACCCATTCAAAAGGCTGTTGAGGAAGGCAGAGCCATGTTTGGAACCATCGACTCTTGGCTCATCTGGTGTATGACCGGAGGAGTCAATGGAGGCATCCATTGCACCGATGTAACAAATGCATGCAGAACGATGCTTTTTAACATCCACTCTTTGGAATGGGATAAAGATCTCTGTGACTTTTTTGAAATTCCAATGAGCATCCTCCCAAATGTATGCAGTTCTTCTGAGATCTATGGCCTGATGACATCAGGGGCCTTGGAAGGCGTGCCAATATCTGGATGTTTGGGGGATCAGTCTGCTGCTTTAGTGGGACAAATGTGTTTCCATGAAGGTCAAGCCAAAAACACATACGGAACAGGCTGTTTTTTACTATGTAATACAGGCCAGAAATGTGTGTTTTCTGAACATGGCCTTCTGACCACATTGGCTTACAAACTAGGCAAAAATAAGCCAGTTTTTTATGCCTT

In [158]:
len('TCTCCTAACTGGCATCACCAGGGAAACAGTCCACTGACCAGTTTCATGGCAGCCTCGAAGCAAACCTCTGCGGGACCGTTGGTAGGAGCGGTGGTCCAGGGAACCAACTCCACTCGCTTTCTAGTTTTCAATAGCAAAACAGCAGAACTAGTTTGCTCTCATCAGGTGGAACTGACACAAGAATACCCAAAAGAAGGATGGGTGGAGCAAGACCCCAAGGAAATCCTTAAATCTGTCTATGAGTGCATAGCGAAAGCTTGTGAAAAACTTGCTGAAGTGAACATTGATATTTCCAACATTAAAGCTATCGGCGTGAGTAACCAGCGGGAAACCACTGTGGTCTGGGACAAGTTCACGGGAGACCCTCTGTACAATGCCGTGGTGTGGCTTGATCTCAGAACCCAGTCTACTGTGGAGACTCTCACTAAGAAAATCCCAGGAAATAGCAACTTTGTCAAGTCTAAGACCGGCCTTCCTCTTAGCACTTACTTCAGTGCTGTGAAGCTGCGGTGGATGCTGGACAACCTAAGACCCATTCAAAAGGCTGTTGAGGAAGGCAGAGCCATGTTTGGAACCATCGACTCTTGGCTCATCTGGTGTATGACCGGAGGAGTCAATGGAGGCATCCATTGCACCGATGTAACAAATGCATGCAGAACGATGCTTTTTAACATCCACTCTTTGGAATGGGATAAAGATCTCTGTGACTTTTTTGAAATTCCAATGAGCATCCTCCCAAATGTATGCAGTTCTTCTGAGATCTATGGCCTGATGACATCAGGGGCCTTGGAAGGCGTGCCAATATCTGGATGTTTGGGGGATCAGTCTGCTGCTTTAGTGGGACAAATGTGTTTCCATGAAGGTCAAGCCAAAAACACATACGGAACAGGCTGTTTTTTACTATGTAATACAGG')

914

In [159]:
#CAGATGAGCCAAGAGTCGATGG

In [160]:
target_base_CT_minus(transcription(base_data['NM_010294::chr5:97602988-97604882']))

['CAAACCTCTGCGGGACCGTTGG',
 'CAAGAATACCCAAAAGAAGG',
 'CAGAACCCAGTCTACTGTGG',
 'CAGTGCTGTGAAGCTGCGGTGG',
 'CAAAAGGCTGTTGAGGAAGG',
 'CAGTTCTTCTGAGATCTATGG',
 'CAATATCTGGATGTTTGGGGG',
 'CAGTCTGCTGCTTTAGTGGG',
 'CAAATGTGTTTCCATGAAGG',
 'CAAGCCAAAAACACATACGG',
 'CAAAAACACATACGGAACAGG',
 'CAGTTTTTTATGCCTTAGAAGG',
 'CGAAATTATAACAACCTCAGGG',
 'CAGTCATTTGCAGGTCGATGG',
 'CAGATATTCTGCACATTCCGG',
 'CAGCCGGAGCTGCAGAAGGGG',
 'CAGAAGGGGTAAATGTTTGG',
 'CAGCAAAGGACCCTGAAAATGG',
 'CAAATACTCCTTAAAATAAAGG']

In [161]:
def CT_target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [162]:
da = CT_target_start(target_base_CT_minus(transcription(base_data['NM_010294::chr5:97602988-97604882'])),transcription(base_data['NM_010294::chr5:97602988-97604882']))

In [163]:
aaa = np.array([1832, 1796, 1730, 1715, 1694, 1685, 1571, 1550, 1508, 1490, 1382, 1355, 1307, 1298, 1205, 1070, 1049, 1028, 977, 857, 734, 695, 641, 635, 581, 542, 536, 434, 392, 380, 374, 341, 311])

In [164]:
aaa

array([1832, 1796, 1730, 1715, 1694, 1685, 1571, 1550, 1508, 1490, 1382,
       1355, 1307, 1298, 1205, 1070, 1049, 1028,  977,  857,  734,  695,
        641,  635,  581,  542,  536,  434,  392,  380,  374,  341,  311])

In [165]:
add_num_list('NM_010294')

[[0, 1894]]

In [166]:
da =1894-np.array(da)-1

In [167]:
da

array([1832, 1715, 1497, 1401, 1355, 1146, 1093, 1070, 1049, 1028, 1023,
        910,  840,  591,  532,  463,  451,  304,   55])

In [168]:
np.intersect1d(aaa,da)

array([1028, 1049, 1070, 1355, 1715, 1832])

In [169]:
transcription(base_data['NM_010294::chr5:97602988-97604882'][:1070])

'AGTCTGCTGCTTTAGTGGGACAAATGTGTTTCCATGAAGGTCAAGCCAAAAACACATACGGAACAGGCTGTTTTTTACTATGTAATACAGGCCAGAAATGTGTGTTTTCTGAACATGGCCTTCTGACCACATTGGCTTACAAACTAGGCAAAAATAAGCCAGTTTTTTATGCCTTAGAAGGTTCTGTTGCTATAGCTGGTGCTGTTATTCGCTGGCTAAGAGACAATTTCGAAATTATAACAACCTCAGGGGAAGTTGAAAATCTTGCAAGAGAGGTAGGCACTTCCTATGGCTGCTACTTTGTCCCAGCCTTTTCGGGGCTGTATGCACCTTACTGGGAACCCAGTGCAAGAGGGATAATCTGTGGTCTCACTCAGTTCACCAACAAATGCCATATTGCCTTTGCTGCGTTAGAAGCTGTTTGTTTCCAAACCCGAGAGATTTTGGATGCCATGAATCGCGACTGTGGAATTCCACTCAGTCATTTGCAGGTCGATGGAGGAATGACCAACAACAGGATTCTTATGCAGCTACAGGCAGATATTCTGCACATTCCGGTAGTTAAGTCTGTCATGCCTGAAACGACGGCCCTGGGAGCTGCCATGGCAGCCGGAGCTGCAGAAGGGGTAAATGTTTGGAGTCTTGAACCTGAAGATTTGTCTACGATCTTGATGGAACGATATGAACCACAAATCCAAGCCACAGAGAGTGAAATTCGTTTTTCAACATGGAAGAGAGCAGTGATGAAGTCAATGGGTTGGGTTACAGCAAAGGACCCTGAAAATGGCGATAACCCTGTCTTCTCTTGTCTGCCCTTGGGTTTTTTTATAGTGAGTAGCATGACGTTATTAATTGGAGCAAGATGTGTCTCAACTGCTGAAGAGTAATTTGAACCACGGACTTGCAATGTGTGAACTTTTTACAGAAAGAATTCCATCTTTTAGTGTGAAACCAATATTATAGGTTTATTTTTACCTATAGGCCACTTGCTTACAGGAA

In [ ]:
97603042, 97603291, 97603438, 97603450, 97603450, 97603519,
       97603578, 97603827, 97603827, 97603897, 97604010, 97604015,
       97604036, 97604057, 97604080, 97604080, 97604133, 97604342,
       97604388, 97604484, 97604702, 97604819

In [308]:
np.array(aa)+np.array(bb)

array([97604820, 97604784, 97604718, 97604703, 97604682, 97604673,
       97604559, 97604538, 97604496, 97604478, 97604370, 97604343,
       97604295, 97604286, 97604193, 97604058, 97604037, 97604016,
       97603965, 97603845, 97603722, 97603683, 97603629, 97603623,
       97603569, 97603530, 97603524, 97603422, 97603380, 97603368,
       97603362, 97603329, 97603299])

In [310]:
np.intersect1d(s,t)

array([97604016, 97604037, 97604058, 97604343, 97604703, 97604820])

In [313]:
len('TACTGTTTTTCTAATGTGTTCTCTTTATTCAAAACCTTTATTTTAAGGAGTATTTGTAGGGGCCTCATGGGTTCCTGTAAGCAAGTGGCCTATAGGTAAAAATAAACCTATAATATTGGTTTCACACTAAAAGATGGAATTCTTTCTGTAAAAAGTTCACACATTGCAAGTCCGTGGTTCAAATTACTCTTCAGCAGTTGAGACACATCTTGCTCCAATTAATAACGTCATGCTACTCACTATAAAAAAACCCAAGGGCAGACAAGAGAAGACAGGGTTATCGCCATTTTCAGGGTCCTTTGCTGTAACCCAACCCATTGACTTCATCACTGCTCTCTTCCATGTTGAAAAACGAATTTCACTCTCTGTGGCTTGGATTTGTGGTTCATATCGTTCCATCAAGATCGTAGACAAATCTTCAGGTTCAAGACTCCAAACATTTACCCCTTCTGCAGCTCCGGCTGCCATGGCAGCTCCCAGGGCCGTCGTTTCAGGCATGACAGACTTAACTACCGGAATGTGCAGAATATCTGCCTGTAGCTGCATAAGAATCCTGTTGTTGGTCATTCCTCCATCGACCTGCAAATGACTGAGTGGAATTCCACAGTCGCGATTCATGGCATCCAAAATCTCTCGGGTTTGGAAACAAACAGCTTCTAACGCAGCAAAGGCAATATGGCATTTGTTGGTGAACTGAGTGAGACCACAGATTATCCCTCTTGCACTGGGTTCCCAGTAAGGTGCATACAGCCCCGAAAAGGCTGGGACAAAGTAGCAGCCATAGGAAGTGCCTACCTCTCTTGCAAGATTTTCAACTTCCCCTGAGGTTGTTATAATTTCGAAATTGTCTCTTAGCCAGCGAATAACAGCACCAGCTATAGCAACAGAACCTTCTAAGGCATAAAAAACTGGCTTATTTTTGCCTAGTTTGTAAGCCAATGTGGTCAGAAGGCCATGTTCAGAAAACACACATTTCTGGCCTGTATTACATAGTAAAAAACAGCCTGTTCCGTATGTGTTTTTGGCTTGACCTTCATGGAAACACATTTGTCCCACTAAAGCAGCAGACTGATCCCCCAAACATCCAGATATTGGCACGCCTTCCAAGGCCCCTGATGTCATCAGGCCATAGATCTCAGAAGAACTGCATACATTTGGGAGGATGCTCATTGGAATTTCAAAAAAGTCACAGAGATCTTTATCCCATTCCAAAGAGTGGATGTTAAAAAGCATCGTTCTGCATGCATTTGTTACATCGGTGCAATGGATGCCTCCATTGACTCCTCCGGTCATACACCAGATGAGCCAAGAGTCGATGGTTCCAAACATGGCTCTGCCTTCCTCAACAGCCTTTT')

1355

In [314]:
97602988+1355

97604343

In [280]:
gene_df[gene_df['geneName']=='Gk2']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
32045,Gk2,NM_010294,chr5,-,97602988,97604882,97603171,97604836,1,"97602988,","97604882,"


In [289]:
transcription('ACACAAGAATACCCAAAAGAAGG')

'CCTTCTTTTGGGTATTCTTGTGT'

In [195]:
base_data['NM_010294::chr5:97602988-97604882']

'TACTGTTTTTCTAATGTGTTCTCTTTATTCAAAACCTTTATTTTAAGGAGTATTTGTAGGGGCCTCATGGGTTCCTGTAAGCAAGTGGCCTATAGGTAAAAATAAACCTATAATATTGGTTTCACACTAAAAGATGGAATTCTTTCTGTAAAAAGTTCACACATTGCAAGTCCGTGGTTCAAATTACTCTTCAGCAGTTGAGACACATCTTGCTCCAATTAATAACGTCATGCTACTCACTATAAAAAAACCCAAGGGCAGACAAGAGAAGACAGGGTTATCGCCATTTTCAGGGTCCTTTGCTGTAACCCAACCCATTGACTTCATCACTGCTCTCTTCCATGTTGAAAAACGAATTTCACTCTCTGTGGCTTGGATTTGTGGTTCATATCGTTCCATCAAGATCGTAGACAAATCTTCAGGTTCAAGACTCCAAACATTTACCCCTTCTGCAGCTCCGGCTGCCATGGCAGCTCCCAGGGCCGTCGTTTCAGGCATGACAGACTTAACTACCGGAATGTGCAGAATATCTGCCTGTAGCTGCATAAGAATCCTGTTGTTGGTCATTCCTCCATCGACCTGCAAATGACTGAGTGGAATTCCACAGTCGCGATTCATGGCATCCAAAATCTCTCGGGTTTGGAAACAAACAGCTTCTAACGCAGCAAAGGCAATATGGCATTTGTTGGTGAACTGAGTGAGACCACAGATTATCCCTCTTGCACTGGGTTCCCAGTAAGGTGCATACAGCCCCGAAAAGGCTGGGACAAAGTAGCAGCCATAGGAAGTGCCTACCTCTCTTGCAAGATTTTCAACTTCCCCTGAGGTTGTTATAATTTCGAAATTGTCTCTTAGCCAGCGAATAACAGCACCAGCTATAGCAACAGAACCTTCTAAGGCATAAAAAACTGGCTTATTTTTGCCTAGTTTGTAAGCCAATGTGGTCAGAAGGCCATGTTCAGAAAACACACATTTCTGGCCTGTATTACATAGTAAAAA

In [199]:
transcription('ACACAAGAATACCCAAAAGAAGG')

'CCTTCTTTTGGGTATTCTTGTGT'

In [312]:
transcription('ATTCAAAAGGCTGTTGAGGAAGG')

'CCTTCCTCAACAGCCTTTTGAAT'

In [ ]:
ACACAAGAATACCCAAAAGA AGG
ATTCAAAAGGCTGTTGAGGA AGG
CCAGATGAGCCAAGAGTCGATGG

In [209]:
gene_df[gene_df['geneName']=='Has3']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
41722,Has3,NM_008217,chr8,+,107596873,107609534,107600539,107605460,4,"107596873,107600539,107603523,107604536,","107597046,107601178,107603625,107609534,"
41723,Has3,NM_001331048,chr8,+,107597467,107609534,107600539,107605460,4,"107597467,107600539,107603523,107604536,","107597583,107601178,107603625,107609534,"
